In [1]:
import os

In [2]:
%pwd

'/Users/vanisuryavanshi/Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vanisuryavanshi/Text-Summarizer'

In [67]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [68]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [73]:
class ConfigurationManager:
    def __init__(
              self, 
              config_filepath: Path=CONFIG_FILE_PATH, 
              params_filepath: Path=PARAMS_FILE_PATH):
          self.config = read_yaml(config_filepath)
          self.params = read_yaml(params_filepath)
          create_directories([self.config.artifacts_root])

    
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL = config.source_URL,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir
          )

          return data_ingestion_config

In [74]:
import os 
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [79]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")   


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        if not os.path.exists(unzip_path):
            os.makedirs(unzip_path)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [80]:
try:
    config = ConfigurationManager(config_filepath=Path("/Users/vanisuryavanshi/Text-Summarizer/config/config.yaml"),
                              params_filepath=Path("/Users/vanisuryavanshi/Text-Summarizer/params.yaml"))
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-03 15:44:39,278: INFO: common: yaml file: /Users/vanisuryavanshi/Text-Summarizer/config/config.yaml loaded successfully]
[2025-01-03 15:44:39,279: INFO: common: yaml file: /Users/vanisuryavanshi/Text-Summarizer/params.yaml loaded successfully]
[2025-01-03 15:44:39,280: INFO: common: Created Directory at: artifacts]
[2025-01-03 15:44:39,280: INFO: common: Created Directory at: artifacts/data_ingestion]
[2025-01-03 15:44:39,281: INFO: 2041518291: File already exists of size: ~ 7718 KB]
